# 중급 튜토리얼: YOLOv5 객체 탐지 모델 컴파일

# DEEPX DX-COM 튜토리얼 (Ubuntu 24.04 + JupyterLab)
이 노트북은 DEEPX **DX-COM v2.0.0**을 사용하여 ONNX 모델을 **.dxnn**으로 컴파일하는 실습을 단계별로 안내합니다.
**중요:** DX-COM 패키지(tar.gz)를 이미 이 작업 디렉터리에 내려받았다고 가정합니다.
압축을 풀면 `./dx_com/` 폴더가 생기며, 그 안에 `dx_com` 실행파일과 샘플이 포함되어 있습니다.

**학습 목표**
- YOLOv5 PyTorch 가중치 → ONNX 내보내기
- YOLO 전처리(JSON) 구성 및 캘리브레이션
- DX-COM 컴파일 및 .dxnn 확인


## 1) 의존 패키지 설치 및 YOLOv5 코드 받기

In [ ]:
# 시스템 라이브러리
!sudo apt-get update -y && sudo apt-get install -y --no-install-recommends libgl1-mesa-glx libglib2.0-0

# 파이썬 패키지
!pip install --quiet torch torchvision onnx

# YOLOv5 레포 및 의존성
!git clone https://github.com/ultralytics/yolov5.git || true
!pip install --quiet -r yolov5/requirements.txt

# 가중치 다운로드 (v7.0 릴리즈 참조)
!wget -q -P yolov5 https://github.com/ultralytics/yolov5/releases/download/v7.0/yolov5s.pt || true

# DX-COM 패키지 해제 (이미 해제했으면 생략)
!ls -1 *.tar.gz || true
!tar -xzf dx_com_M1_v2.0.0.tar.gz || echo '이미 해제했거나 파일명을 맞춰주세요.'

## 2) YOLOv5s를 ONNX로 익스포트

In [ ]:
# export.py 사용하여 ONNX 내보내기
!python yolov5/export.py --weights yolov5/yolov5s.pt --img 640 --batch 1 --include onnx

# 산출물 확인 및 이동(선택)
!ls -lh yolov5/*.onnx

## 3) DX-COM JSON 구성 (YOLO 입력/전처리/캘리브레이션)

In [ ]:
import json, os

yolo_cfg = {
  "inputs": {"images": [1,3,640,640]},
  "calibration_num": 5,
  "calibration_method": "ema",
  "default_loader": {
    "dataset_path": "./yolov5/data/images",
    "file_extensions": ["jpg","jpeg","png"],
    "preprocessings": [
      {"convertColor": {"form": "BGR2RGB"}},
      {"resize": {"width": 640, "height": 640, "interpolation": "LINEAR"}},
      {"div": {"x": 255.0}}
    ]
  }
}
with open("yolov5s.json","w") as f:
    json.dump(yolo_cfg,f,indent=2)
print("✅ JSON 저장: yolov5s.json")
os.system("sed -n '1,120p' yolov5s.json")

## 4) DX-COM으로 컴파일 실행

In [ ]:
# 출력 폴더
!mkdir -p dx_com/output/yolov5s

# YOLOv5s ONNX 경로
!test -f yolov5/yolov5s.onnx && echo 'onnx ok' || echo 'yolov5s.onnx 경로를 확인하세요.'

# 컴파일
!./dx_com/dx_com -m yolov5/yolov5s.onnx -c yolov5s.json -o dx_com/output/yolov5s || echo 'DX-COM 실행 실패시 경로/권한/의존성 확인'

# 결과 확인
!ls -lh dx_com/output/yolov5s

## 5) 팁 & 주의
- NMS는 ONNX에 포함하지 않는 것을 권장 (후처리는 앱/런타임에서 수행)
- Letterbox를 정확히 재현하려면 고급 튜토리얼의 custom loader 접근 검토
- 정확도 튜닝: calibration 데이터 다양화, 수량 증가, ema vs minmax 비교
